In [8]:
import requests
import json
import pandas as pd
import re
from pprint import pprint

# Import the API key
from config import ombdapi_key

In [10]:
# read oscar winning movie list
oscar_df = pd.DataFrame( pd.read_csv("../project1-team8/Resources/oscar.csv", index_col='Unnamed: 0') )

# remove blank film names
oscar_df.dropna(inplace=True)

In [11]:
# limit to row. daily API allow to run for 1000
oscar_df = oscar_df.loc[0:900,:]

# rename oscar df to avoide overlap with imdb data
for name in oscar_df.columns :
    oscar_df = oscar_df.rename( columns={ name : f'Oscar_{name}' })
    
oscar_df = oscar_df.reset_index(drop=True) # when select partial for API
oscar_df

,Oscar_Film,Oscar_Year,Oscar_Award,Oscar_Nomination
0,Everything Everywhere All at Once,2022,7,11
1,All Quiet on the Western Front,2022,4,9
2,The Whale,2022,2,3
3,Top Gun: Maverick,2022,1,6
4,Black Panther: Wakanda Forever,2022,1,5
...,...,...,...,...
896,Casals Conducts: 1964,1964,1,1
897,The Cat Concerto,1946,1,1
898,Cavalcade,33,3,4
899,Chagall,1963,1,1


In [3]:
# omdb api preparation
base_url = "http://www.omdbapi.com/?"
params = { "apikey" : ombdapi_key, "type" : "movie" }

# general info to extract from api. column names are key in api.
general_col = ['Year', 'Rated', 'Runtime', 'Genre', 'Director', 'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', \
               'imdbRating', 'imdbVotes','BoxOffice']

for i in range(len(general_col)) :
    oscar_df[ general_col[i] ] = None

# non omdb rating to fulfill
ratings_col = ['Ratings_IMD', 'Ratings_Rotten_Tomatoes' ,'Ratings_Metacritic']

for i in range(len(ratings_col)) :
    oscar_df[ ratings_col[i] ] = None


In [4]:
# request API

for index in range(len(oscar_df)) :
    
    # export movie by film name in oscar df
    params["t"] = re.sub('\s+', '+', oscar_df.loc[ index , 'Oscar_Film' ] ) 
    movie_data = requests.get( base_url , params ).json()
    
    # extract general info and fulfill the dataframe, by colnames = keys
    for info_col in general_col :
        
        try :
            oscar_df.loc[ index , info_col ] = movie_data[ info_col ]
            
        except KeyError :
            oscar_df.loc[ index , info_col ] = None
            print( f'Film: {params["t"]}. Missing Key: {info_col}')
    
    # extract other ratings and fill rating columns
    try :
        temp_rating = pd.DataFrame( movie_data[ 'Ratings' ] ).set_index("Source")
    except KeyError :
        print( f'Film: {params["t"]}. Missing Other Ratings')
        oscar_df.loc[ index , 'Ratings_IMD'] = None
        oscar_df.loc[ index , 'Ratings_Rotten_Tomatoes'] = None
        oscar_df.loc[ index , 'Ratings_Metacritic'] = None
        
    try :
        oscar_df.loc[ index , 'Ratings_IMD'] = temp_rating.loc[ "Internet Movie Database" ,'Value']
    except KeyError :
        oscar_df.loc[ index , 'Ratings_IMD'] = None
        
    try :
        oscar_df.loc[ index , 'Ratings_Rotten_Tomatoes'] = temp_rating.loc[ "Rotten Tomatoes" ,'Value']
    except KeyError :
        oscar_df.loc[ index , 'Ratings_Rotten_Tomatoes'] = None
        
    try :
        oscar_df.loc[ index , 'Ratings_Metacritic'] = temp_rating.loc["Metacritic" ,'Value']
    except KeyError :
        oscar_df.loc[ index , 'Ratings_Metacritic'] = None


Film: Period.+End+of+Sentence.. Missing Key: Year 
Film: Period.+End+of+Sentence.. Missing Key: Rated 
Film: Period.+End+of+Sentence.. Missing Key: Runtime 
Film: Period.+End+of+Sentence.. Missing Key: Genre 
Film: Period.+End+of+Sentence.. Missing Key: Director 
Film: Period.+End+of+Sentence.. Missing Key: Writer 
Film: Period.+End+of+Sentence.. Missing Key: Actors 
Film: Period.+End+of+Sentence.. Missing Key: Plot 
Film: Period.+End+of+Sentence.. Missing Key: Language 
Film: Period.+End+of+Sentence.. Missing Key: Country 
Film: Period.+End+of+Sentence.. Missing Key: Awards 
Film: Period.+End+of+Sentence.. Missing Key: imdbRating 
Film: Period.+End+of+Sentence.. Missing Key: imdbVotes 
Film: Period.+End+of+Sentence.. Missing Key: BoxOffice 
Film: Flesh+and+Sand. Missing Key: Year 
Film: Flesh+and+Sand. Missing Key: Rated 
Film: Flesh+and+Sand. Missing Key: Runtime 
Film: Flesh+and+Sand. Missing Key: Genre 
Film: Flesh+and+Sand. Missing Key: Director 
Film: Flesh+and+Sand. Missing Key:

Film: The+Ten-Year+Lunch. Missing Key: Year 
Film: The+Ten-Year+Lunch. Missing Key: Rated 
Film: The+Ten-Year+Lunch. Missing Key: Runtime 
Film: The+Ten-Year+Lunch. Missing Key: Genre 
Film: The+Ten-Year+Lunch. Missing Key: Director 
Film: The+Ten-Year+Lunch. Missing Key: Writer 
Film: The+Ten-Year+Lunch. Missing Key: Actors 
Film: The+Ten-Year+Lunch. Missing Key: Plot 
Film: The+Ten-Year+Lunch. Missing Key: Language 
Film: The+Ten-Year+Lunch. Missing Key: Country 
Film: The+Ten-Year+Lunch. Missing Key: Awards 
Film: The+Ten-Year+Lunch. Missing Key: imdbRating 
Film: The+Ten-Year+Lunch. Missing Key: imdbVotes 
Film: The+Ten-Year+Lunch. Missing Key: BoxOffice 
Film: Anna+&amp;+Bella. Missing Key: Year 
Film: Anna+&amp;+Bella. Missing Key: Rated 
Film: Anna+&amp;+Bella. Missing Key: Runtime 
Film: Anna+&amp;+Bella. Missing Key: Genre 
Film: Anna+&amp;+Bella. Missing Key: Director 
Film: Anna+&amp;+Bella. Missing Key: Writer 
Film: Anna+&amp;+Bella. Missing Key: Actors 
Film: Anna+&amp;+B

Film: Black+Fox:+The+Rise+and+Fall+of+Adolf+Hitler. Missing Key: Year 
Film: Black+Fox:+The+Rise+and+Fall+of+Adolf+Hitler. Missing Key: Rated 
Film: Black+Fox:+The+Rise+and+Fall+of+Adolf+Hitler. Missing Key: Runtime 
Film: Black+Fox:+The+Rise+and+Fall+of+Adolf+Hitler. Missing Key: Genre 
Film: Black+Fox:+The+Rise+and+Fall+of+Adolf+Hitler. Missing Key: Director 
Film: Black+Fox:+The+Rise+and+Fall+of+Adolf+Hitler. Missing Key: Writer 
Film: Black+Fox:+The+Rise+and+Fall+of+Adolf+Hitler. Missing Key: Actors 
Film: Black+Fox:+The+Rise+and+Fall+of+Adolf+Hitler. Missing Key: Plot 
Film: Black+Fox:+The+Rise+and+Fall+of+Adolf+Hitler. Missing Key: Language 
Film: Black+Fox:+The+Rise+and+Fall+of+Adolf+Hitler. Missing Key: Country 
Film: Black+Fox:+The+Rise+and+Fall+of+Adolf+Hitler. Missing Key: Awards 
Film: Black+Fox:+The+Rise+and+Fall+of+Adolf+Hitler. Missing Key: imdbRating 
Film: Black+Fox:+The+Rise+and+Fall+of+Adolf+Hitler. Missing Key: imdbVotes 
Film: Black+Fox:+The+Rise+and+Fall+of+Adolf+

Film: Forbidden+Games. Missing Key: Year 
Film: Forbidden+Games. Missing Key: Rated 
Film: Forbidden+Games. Missing Key: Runtime 
Film: Forbidden+Games. Missing Key: Genre 
Film: Forbidden+Games. Missing Key: Director 
Film: Forbidden+Games. Missing Key: Writer 
Film: Forbidden+Games. Missing Key: Actors 
Film: Forbidden+Games. Missing Key: Plot 
Film: Forbidden+Games. Missing Key: Language 
Film: Forbidden+Games. Missing Key: Country 
Film: Forbidden+Games. Missing Key: Awards 
Film: Forbidden+Games. Missing Key: imdbRating 
Film: Forbidden+Games. Missing Key: imdbVotes 
Film: Forbidden+Games. Missing Key: BoxOffice 


TypeError: expected string or bytes-like object

In [5]:
oscar_df

,Oscar_Film,Oscar_Year,Oscar_Award,Oscar_Nomination,Year,Rated,Runtime,Genre,Director,Writer,...,Plot,Language,Country,Awards,imdbRating,imdbVotes,BoxOffice,Ratings_IMD,Ratings_Rotten_Tomatoes,Ratings_Metacritic
0,Everything Everywhere All at Once,2022,7,11,2022,R,139 min,"Action, Adventure, Comedy","Daniel Kwan, Daniel Scheinert","Daniel Kwan, Daniel Scheinert",...,A middle-aged Chinese immigrant is swept up in...,"English, Mandarin, Cantonese",United States,Won 7 Oscars. 372 wins & 357 nominations total,7.8,"445,131","$77,191,785",7.8/10,93%,81/100
1,All Quiet on the Western Front,2022,4,9,2022,R,148 min,"Action, Drama, War",Edward Berger,"Edward Berger, Lesley Paterson, Ian Stokell",...,A young German soldier's terrifying experience...,"German, French","Germany, United States, United Kingdom",Won 4 Oscars. 48 wins & 81 nominations total,7.8,"205,354",N/A,7.8/10,90%,76/100
2,The Whale,2022,2,3,2022,R,117 min,Drama,Darren Aronofsky,Samuel D. Hunter,...,"A reclusive, morbidly obese English teacher at...",English,United States,Won 2 Oscars. 43 wins & 116 nominations total,7.7,"139,972","$17,463,630",7.7/10,64%,60/100
3,Top Gun: Maverick,2022,1,6,2022,PG-13,130 min,"Action, Drama",Joseph Kosinski,"Jim Cash, Jack Epps Jr., Peter Craig",...,"After thirty years, Maverick is still pushing ...",English,United States,Won 1 Oscar. 93 wins & 214 nominations total,8.3,"590,794","$718,732,821",8.3/10,96%,78/100
4,Black Panther: Wakanda Forever,2022,1,5,2022,PG-13,161 min,"Action, Adventure, Drama",Ryan Coogler,"Ryan Coogler, Joe Robert Cole, Stan Lee",...,The people of Wakanda fight to protect their h...,"English, Xhosa, Maya, Spanish, Haitian, French",United States,Won 1 Oscar. 44 wins & 154 nominations total,6.7,"270,203","$453,829,060",6.7/10,84%,67/100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1355,The Yankee Doodle Mouse,1943,1,1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1356,The Yearling,1946,2,7,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1357,"Yesterday, Today and Tomorrow",1964,1,1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1358,You Can't Take It with You,1938,2,7,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [6]:
oscar_df.to_csv('.\Resources\oscar_omdb.csv', index=False)